In [ ]:
#importing the needed datasets
import pandas as pd
import numpy as np
import lightgbm as lgb
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [2]:

#importing dataset
dataset=pd.read_csv('C:/Users/BadBoy/Desktop/kaggle2/data.csv',low_memory=False)
dataset=dataset.drop("Instance",axis=1)
testset=pd.read_csv('C:/Users/BadBoy/Desktop/kaggle2/test.csv',low_memory=False)
testset=testset.drop("Instance",axis=1)
#merge=pd.concat([dataset,testset])
dataset.Gender.unique()

#renaming the columns for convenience

dataset = dataset.rename(columns={'Year of Record': 'Year'})
dataset = dataset.rename(columns={'Yearly Income in addition to Salary (e.g. Rental Income)': 'Extra'})
dataset = dataset.rename(columns={'Work Experience in Current Job [years]': 'Experience'})
dataset = dataset.rename(columns={'Crime Level in the City of Employement':'Crime'})
dataset= dataset.rename(columns={'Total Yearly Income [EUR]':'Income'})
dataset= dataset.rename(columns={'Body Height [cm]':'Height'})

testset = testset.rename(columns={'Year of Record': 'Year'})
testset = testset.rename(columns={'Yearly Income in addition to Salary (e.g. Rental Income)': 'Extra'})
testset = testset.rename(columns={'Work Experience in Current Job [years]': 'Experience'})
testset = testset.rename(columns={'Crime Level in the City of Employement':'Crime'})
testset = testset.rename(columns={'Total Yearly Income [EUR]':'Income'})
testset = testset.rename(columns={'Body Height [cm]':'Height'})

#scaling the Income column  with log
dataset['Income'] = dataset['Income'].apply(np.log)

#converting 'f' and 'female' into 'female'

dataset['Gender'].replace(['f','female'],'female',inplace=True)
testset['Gender'].replace(['f','female'],'female',inplace=True)

#changing Extra column to int by removing EUR
dataset['Extra']=dataset['Extra'].str.replace('\D+','').astype(int)
testset['Extra']=testset['Extra'].str.replace('\D+','').astype(int)

#finidng if there is any null value in the dataset
print(dataset.isnull().sum())

#dropping if both dataset and preofession is a null value
index_to_drop = dataset[((dataset.Gender == 'unknown') | (pd.isnull(dataset.Gender))) & ((pd.isnull(dataset.Profession)))].index
dataset.drop(index_to_drop,inplace= True)
print(dataset.shape)

y = dataset['Income']
y=pd.DataFrame(y)

# concatinating the dataset and test set into one dataset
combine=pd.concat([dataset, testset])
y_only = combine['Income']
y_only=pd.DataFrame(y_only)

# dropping the income columns from the dataset and combining it
train_model = dataset.drop(columns =['Income'])
test_model =  testset.drop(columns =['Income'])                                                     
combined_df = pd.concat([train_model, test_model])
print(combined_df.isnull().sum())

print(train_model.shape)
print(test_model.shape)
print(combined_df.shape)
print(y.shape)
print(y_only.shape)


#preprocessing data and converting all the nan values to 'Missing'
imp_gender = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value='Missing')
imp_Satisfation = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value='Missing')
imp_university = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value='Missing')
imp_profession = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value='Missing')


imp_Satisfation = imp_Satisfation.fit(combined_df[['Satisfation with employer']])
combined_df['Satisfation with employer'] = imp_Satisfation.transform(combined_df[['Satisfation with employer']]).ravel()

imp_gender = imp_gender.fit(combined_df[['Gender']])
combined_df['Gender'] = imp_gender.transform(combined_df[['Gender']]).ravel()

imp_university = imp_university.fit(combined_df[['University Degree']])
combined_df['University Degree'] = imp_university.transform(combined_df[['University Degree']]).ravel()

imp_profession = imp_university.fit(combined_df[['Profession']])
combined_df['Profession'] = imp_university.transform(combined_df[['Profession']]).ravel()


#replacing the "0",'other','unknown' with 'missing' for the categorical columns
combined_df['Gender'].replace(['0','other','unknown'],'Missing',inplace = True)
combined_df['University Degree'].replace('0','No'],'Missing',inplace = True)
combined_df['Housing Situation'].replace(['0','nA'],'Missing',inplace = True)
#converting th e column into float and calulating the mean
avg_year = combined_df["Year"].astype("float").mean(axis=0)
#converting th e column into float and calculating the mean
avg_age = combined_df["Age"].astype("float").mean(axis=0)

#replacing nan values by average.
combined_df["Year"].replace(np.nan, avg_year, inplace=True)
combined_df["Age"].replace(np.nan, avg_age, inplace=True)


combined_df.Experience.replace(["#NUM!"], np.nan, inplace=True)
combined_df['Experience']=combined_df['Experience'].fillna(combined_df.Experience.median())
avg_exp = combined_df["Experience"].astype("float").mean(axis=0)
#taking into concideration only those countries which have been metioned more than 100 times and the age is less than 100.
#The countries which are present less than 100 times marking the, as notimportant.
countries_greaterthan100 = combined_df.Country.value_counts()[combined_df.Country.value_counts() > 100]
countrylist = countries_greaterthan100.index.tolist()
combined_df.Country.where(combined_df.Country.isin(countrylist),'notimportant',inplace = True)
combined_df.Age.where(combined_df.Age <= 100,inplace = True)






Year                          4013
Housing Situation                0
Crime                            0
Experience                       0
Satisfation with employer    38087
Gender                       74127
Age                              0
Country                          0
Size of City                     0
Profession                    2853
University Degree            80600
Wears Glasses                    0
Hair Color                   70211
Height                           0
Extra                            0
Income                           0
dtype: int64
(1048187, 16)
Year                           5492
Housing Situation                 0
Crime                             0
Experience                        0
Satisfation with employer     51485
Gender                       100244
Age                               0
Country                           1
Size of City                      0
Profession                     3507
University Degree            109309
Wears Glasses    

In [ ]:
#dropping few columns
combined_df=combined_df.drop(['Wears Glasses','Hair Color'],axis=1)

In [5]:
#concating the columns to increase the number of features
def create_cat_con(df,cats,cons,normalize=True):
    for i,cat in enumerate(cats):
        vc = df[cat].value_counts(dropna=False, normalize=normalize).to_dict()
        nm = cat + '_FE_FULL'
        df[nm] = df[cat].map(vc)
        df[nm] = df[nm].astype('float32')
        for j,con in enumerate(cons):
#             print("cat %s con %s"%(cat,con))
            new_col = cat +'_'+ con
            print('timeblock frequency encoding:', new_col)
            df[new_col] = df[cat].astype(str)+'_'+df[con].astype(str)
            temp_df = df[new_col]
            fq_encode = temp_df.value_counts(normalize=True).to_dict()
            df[new_col] = df[new_col].map(fq_encode)
            df[new_col] = df[new_col]/df[cat+'_FE_FULL']
    return df

cats = ['Year', 'Country',
        'Profession', 'University Degree',
        'Satisfation with employer','Housing Situation','Gender' ]

cons = ['Size of City','Height','Experience', 'Crime','Extra',]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

#performing targert encoding on the categorical columns
combined_df = create_cat_con(combined_df,cats,cons)

feat_le =ce.TargetEncoder(cols=['Country','Profession','University Degree','Satisfation with employer','Gender'])
feat_le.fit(combined_df,y_only)
combined_df = feat_le.transform(combined_df)



timeblock frequency encoding: Year_Size of City
timeblock frequency encoding: Year_Height
timeblock frequency encoding: Year_Experience
timeblock frequency encoding: Year_Crime
timeblock frequency encoding: Year_Extra
timeblock frequency encoding: Country_Size of City
timeblock frequency encoding: Country_Height
timeblock frequency encoding: Country_Experience
timeblock frequency encoding: Country_Crime
timeblock frequency encoding: Country_Extra
timeblock frequency encoding: Profession_Size of City
timeblock frequency encoding: Profession_Height
timeblock frequency encoding: Profession_Experience
timeblock frequency encoding: Profession_Crime
timeblock frequency encoding: Profession_Extra
timeblock frequency encoding: University Degree_Size of City
timeblock frequency encoding: University Degree_Height
timeblock frequency encoding: University Degree_Experience
timeblock frequency encoding: University Degree_Crime
timeblock frequency encoding: University Degree_Extra
timeblock frequenc

In [6]:
#scaling the data using min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
combined_df = pd.DataFrame(scaler.fit_transform(combined_df), columns=combined_df.columns, index=combined_df.index)

In [7]:
features_col =  list(set(combined_df))
features_col

['Country_FE_FULL',
 'Satisfation with employer_Extra',
 'Country_Extra',
 'Housing Situation_Experience',
 'Age_Crime',
 'Hair Color',
 'University Degree_Extra',
 'Profession_FE_FULL',
 'Profession_Crime',
 'Satisfation with employer',
 'Housing Situation_Crime',
 'Year_Size of City',
 'Age_Height',
 'Experience',
 'Country_Experience',
 'Year_Experience',
 'Satisfation with employer_Height',
 'Age_Experience',
 'University Degree_FE_FULL',
 'Satisfation with employer_Size of City',
 'University Degree_Size of City',
 'Age_FE_FULL',
 'Year',
 'Housing Situation_Height',
 'Housing Situation',
 'Country_Crime',
 'Country_Height',
 'University Degree_Height',
 'Year_FE_FULL',
 'Crime',
 'Housing Situation_Size of City',
 'Country',
 'Age',
 'Height',
 'Profession_Experience',
 'University Degree_Crime',
 'Housing Situation_FE_FULL',
 'Profession',
 'Year_Extra',
 'Gender',
 'Country_Size of City',
 'Satisfation with employer_FE_FULL',
 'Profession_Height',
 'University Degree_Experience

In [8]:
#splitting the dataset and training the model
X_train,X_test = combined_df[features_col].iloc[:1048187],combined_df[features_col].iloc[1048187:]
Y_train=dataset['Income'].iloc[:1048187]
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size=0.2,random_state=1234)
params = {
          'max_depth': 20,
          'learning_rate': 0.001,
          "boosting": "gbdt",
          "bagging_seed": 11,
          "metric": 'mse',
          "verbosity": -1,
         }
trn_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_val, label=y_val)
# test_data = lgb.Dataset(X_test)
clf = lgb.train(params, trn_data, 100000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
#predicting using our model
pre_test_lgb = clf.predict(X_test)
'done'
#calulating the mean squared error
from sklearn.metrics import mean_squared_error
pre_val_lgb = clf.predict(x_val)
val_mse = mean_squared_error(np.exp(y_val),np.exp(pre_val_lgb))
val_mse

Training until validation scores don't improve for 500 rounds.
[1000]	training's l2: 0.60333	valid_1's l2: 0.606496
[2000]	training's l2: 0.202952	valid_1's l2: 0.204691
[3000]	training's l2: 0.117629	valid_1's l2: 0.118838
[4000]	training's l2: 0.0901986	valid_1's l2: 0.0912165
[5000]	training's l2: 0.0768179	valid_1's l2: 0.0777134
[6000]	training's l2: 0.069296	valid_1's l2: 0.0701322
[7000]	training's l2: 0.0640748	valid_1's l2: 0.0648782
[8000]	training's l2: 0.0604983	valid_1's l2: 0.0612956
[9000]	training's l2: 0.0580744	valid_1's l2: 0.0588705
[10000]	training's l2: 0.0562116	valid_1's l2: 0.0570278
[11000]	training's l2: 0.0548781	valid_1's l2: 0.0557061
[12000]	training's l2: 0.0537774	valid_1's l2: 0.0546312
[13000]	training's l2: 0.0527368	valid_1's l2: 0.0536367
[14000]	training's l2: 0.0516844	valid_1's l2: 0.0526322
[15000]	training's l2: 0.0509648	valid_1's l2: 0.0519869
[16000]	training's l2: 0.0503313	valid_1's l2: 0.0514177
[17000]	training's l2: 0.0497911	valid_1's

518526825.4462467

In [12]:
import math
math.sqrt(val_mse)

22771.184102857864

In [13]:
#uploading the result into the csv file
submit=pd.read_csv('C:/Users/BadBoy/Desktop/kaggle2/submit.csv',low_memory=False)
solution= pd.DataFrame ({'Instance':  submit['Instance'],'Total Yearly Income [EUR]': np.exp(pre_test_lgb)})
solution.to_csv(r'C:/Users/BadBoy/Desktop/kaggle2/sol_best.csv'', index=False)